# 📊 US Accidents – Initial EDA with DuckDB

This notebook uses DuckDB to efficiently analyze a large CSV file (~3GB) without loading it all into memory.

In [1]:
# Install DuckDB if not already
!pip install duckdb --quiet

In [2]:
import duckdb
import pandas as pd

In [24]:
# Set the path to your dataset (adjust if needed)
csv_path = '../data/US_Accidents_March23.csv'


## View Column Names

In [13]:
df_columns = duckdb.query(f"""
    SELECT * FROM read_csv_auto('{csv_path}', SAMPLE_SIZE=-1)
    LIMIT 5
""").df()

print(df_columns.columns)


Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')


**Observations:**

The dataset contains 47 columns describing the time, location, severity, weather, and environmental conditions of each reported traffic accident. Key variables include:

ID, Start_Time, End_Time: Unique ID and timestamps of the accident

Severity: Categorical scale (1–4) indicating seriousness

Start_Lat, Start_Lng, End_Lat, End_Lng: GPS coordinates

Weather_Condition, Temperature(F), Humidity(%): Weather context at the time

City, County, State, Zipcode: Geographic identifiers

Traffic_Signal, Junction, Crossing, etc.: Road infrastructure conditions

Sunrise_Sunset, Civil_Twilight, etc.: Time-of-day indicators (day vs night)

This metadata will help guide our risk metric computations and exploratory visualizations later.

## Count Total Number of Rows

In [14]:
duckdb.query(f"""
    SELECT COUNT(*) AS total_rows
    FROM read_csv_auto('{csv_path}', SAMPLE_SIZE=-1)
""").show()


┌────────────┐
│ total_rows │
│   int64    │
├────────────┤
│    7728394 │
└────────────┘



**Observations:** There are 7728394 rows in this dataset, so almost 7.7 million rows. It is a huge dataset, so ideal for meaningful statistical analysis.

## Summary stats for numeric columns

In [15]:
duckdb.query(f"""
    SELECT 
        COUNT(*) AS total_rows,
        MIN("Severity") AS min_severity,
        MAX("Severity") AS max_severity,
        AVG("Severity") AS avg_severity,
        MIN("Distance(mi)") AS min_distance,
        MAX("Distance(mi)") AS max_distance,
        AVG("Distance(mi)") AS avg_distance
    FROM read_csv_auto('{csv_path}', SAMPLE_SIZE=-1)
""").df()


,total_rows,min_severity,max_severity,avg_severity,min_distance,max_distance,avg_distance
0,7728394,1,4,2.212384,0.0,441.75,0.561842


**Observations:** Among 7.7 million accidents, the average severity is 2.21 on a 1–4 scale, and while most accidents cover short distances (avg 0.56 mi), some span as far as 441.75 miles, indicating rare but extreme cases.

## Previewing a Random 100K Sample of the Dataset


In [10]:
df_sample = duckdb.query(f"""
SELECT * FROM read_csv_auto('{csv_path}', SAMPLE_SIZE=-1)
USING SAMPLE 100000
""").df()

df_sample.head()


,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1780007,Source2,2,2019-10-26 20:13:33,2019-10-26 20:45:00,29.780870,-95.340271,NaN,NaN,1.180,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-9202,Source2,2,2016-12-31 21:45:25,2016-12-31 22:33:38,37.550552,-122.022476,NaN,NaN,0.000,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-1841314,Source2,2,2019-11-24 10:24:34,2019-11-24 10:54:22,37.523712,-77.528381,NaN,NaN,0.000,...,False,False,False,False,False,False,Day,Day,Day,Day
3,A-118170,Source2,3,2016-12-15 10:29:41,2016-12-15 10:58:53,26.136631,-80.169502,NaN,NaN,0.010,...,False,False,False,False,False,False,Day,Day,Day,Day
4,A-4170441,Source1,2,2022-05-15 15:40:36,2022-05-15 19:28:30,40.806894,-73.924426,40.81538,-73.930813,0.675,...,False,False,False,False,True,False,Day,Day,Day,Day


**Observations:** The dataset contains detailed records of traffic accidents, with timestamped location data, severity levels, and various contextual attributes such as lighting conditions and road features. Notably, many entries have missing values for End_Lat and End_Lng, indicating potential incompleteness in geographic end-point data.

In [12]:
duckdb.query(f"""
SELECT
  MIN(Start_Time) as min_time,
  MAX(Start_Time) as max_time,
  COUNT(*) as total_rows
FROM '{csv_path}'
""").df()

,min_time,max_time,total_rows
0,2016-01-14 20:18:33,2023-03-31 23:30:00,7728394


**Observations:** The dataset spans over 7.7 million records of traffic accidents collected between January 14, 2016 and March 31, 2023. This wide temporal coverage will help us to do robust longitudinal analysis of accident trends and patterns across years and seasons.

## Missing value count per column

In [18]:
df = duckdb.query(f"""
    SELECT * FROM read_csv_auto('{csv_path}', SAMPLE_SIZE=-1)
""").df()

# Now count missing values per column
missing = df.isnull().sum().sort_values(ascending=False)
print(missing[missing > 0])


End_Lat                  3402762
End_Lng                  3402762
Precipitation(in)        2203586
Wind_Chill(F)            1999019
Wind_Speed(mph)           571233
Visibility(mi)            177098
Wind_Direction            175206
Humidity(%)               174144
Weather_Condition         173459
Temperature(F)            163853
Pressure(in)              140679
Weather_Timestamp         120228
Nautical_Twilight          23246
Civil_Twilight             23246
Sunrise_Sunset             23246
Astronomical_Twilight      23246
Airport_Code               22635
Street                     10869
Timezone                    7808
Zipcode                     1915
City                         253
Description                    5
dtype: int64


**Observations:** Several columns contain substantial missing data, especially weather-related fields such as Precipitation(in), Wind_Chill(F), and Temperature(F), each missing over 1 million entries. Additionally, End_Lat and End_Lng are missing in more than 3.4 million records, indicating potential limitations in spatial resolution for post-incident locations.

## Unique value count for selected categorical columns

In [19]:
duckdb.query(f"""
    SELECT 
        COUNT(DISTINCT "Source") AS unique_sources,
        COUNT(DISTINCT "State") AS unique_states,
        COUNT(DISTINCT "Weather_Condition") AS unique_weather_conditions
    FROM read_csv_auto('{csv_path}', SAMPLE_SIZE=-1)
""").df()


,unique_sources,unique_states,unique_weather_conditions
0,3,49,144


**Observations:** The dataset includes data from 3 unique sources and covers 49 U.S. states, indicating broad geographic coverage. Additionally, there are 144 distinct weather conditions, suggesting rich diversity in environmental context across incidents.

Now, I want to explore, out of curiosity, which state is missing from this dataset.

In [20]:
import duckdb

duckdb.query(f"""
    SELECT DISTINCT State
    FROM read_csv_auto('{csv_path}', SAMPLE_SIZE=-1)
    ORDER BY State
""").df()


,State
0,AL
1,AR
2,AZ
3,CA
4,CO
5,CT
6,DC
7,DE
8,FL
9,GA


**Observations:** From the list of states for which we have data, we noticed that we have datasets for all the US states plus DC, except for two states: for HI (Hawaii) and AK (Alaska).

## Data Types Summary

In [25]:
import duckdb
import pandas as pd


# Step 1: Load only the first 500,000 rows into a Pandas DataFrame
df_sample = duckdb.query(f"""
    SELECT * FROM read_csv_auto('{csv_path}', SAMPLE_SIZE=-1)
    USING SAMPLE 500000
""").df()

# Step 2: Filter numeric columns only
numeric_df = df_sample.select_dtypes(include='number')

# Step 3: Run .describe() on numeric columns
numeric_df.describe()


,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,500000.000000,500000.000000,500000.000000,38008.000000,38008.000000,500000.000000,492296.000000,108987.000000,491637.000000,493810.000000,489335.000000,418366.000000,98180.000000
mean,2.349868,35.993022,-97.020339,36.284157,-95.759884,0.136064,64.416718,45.451277,64.723475,29.936715,9.205086,8.686872,0.037477
std,0.495791,5.033997,18.180317,5.262490,18.125825,0.999913,17.238032,24.368842,21.580408,0.476214,2.228793,5.515389,0.402597
min,1.000000,24.661196,-124.494294,24.660821,-124.494360,0.000000,-77.800000,-40.000000,2.000000,0.120000,0.000000,0.000000,0.000000
25%,2.000000,32.839848,-118.031665,33.462340,-117.810396,0.000000,54.000000,28.000000,49.000000,29.880000,10.000000,5.800000,0.000000
50%,2.000000,34.487602,-95.394310,36.220026,-88.059938,0.010000,66.200000,39.900000,66.000000,29.990000,10.000000,8.100000,0.000000
75%,3.000000,40.161207,-81.388466,40.205698,-80.243289,0.010000,77.000000,67.000000,83.000000,30.100000,10.000000,11.500000,0.010000
max,4.000000,49.000269,-67.841889,48.999132,-67.842169,90.370003,167.000000,114.000000,100.000000,58.630000,111.000000,822.800000,10.140000


**Observations:** The severity of accidents in the sample mostly centers around level 2, with a range from 1 to 4, indicating a predominance of moderate incidents. Most accidents occurred within short travel distances (median ~0.01 miles), although rare cases involved distances over 90 miles. The weather-related features show wide variation, with temperature ranging from -77.8°F to 167°F and wind speeds reaching up to 822.8 mph, suggesting possible outliers or data entry errors that should be carefully examined during cleaning.

## Inspect column types from the full dataset

In [26]:
import duckdb

# Get column names and types using DuckDB's DESCRIBE function
df_schema = duckdb.query(f"""
    DESCRIBE SELECT * FROM read_csv_auto('{csv_path}', SAMPLE_SIZE=-1)
""").df()

# Display the result
df_schema.head(50)  # View all ~46 columns and types


,column_name,column_type,null,key,default,extra
0,ID,VARCHAR,YES,None,None,None
1,Source,VARCHAR,YES,None,None,None
2,Severity,BIGINT,YES,None,None,None
3,Start_Time,TIMESTAMP,YES,None,None,None
4,End_Time,TIMESTAMP,YES,None,None,None
5,Start_Lat,DOUBLE,YES,None,None,None
6,Start_Lng,DOUBLE,YES,None,None,None
7,End_Lat,DOUBLE,YES,None,None,None
8,End_Lng,DOUBLE,YES,None,None,None
9,Distance(mi),DOUBLE,YES,None,None,None


**Observations:** The dataset contains a wide range of variable types, including timestamps (e.g., Start_Time, End_Time), continuous numerical values (e.g., Temperature(F), Distance(mi)), and categorical strings (e.g., City, Weather_Condition). Several columns also use boolean flags to indicate the presence of infrastructure features such as Railway, Traffic_Signal, and Junction. Importantly, all columns are nullable, indicating that missing data is present across all fields and must be addressed during preprocessing.

## Top 10 Accident-prone States

In [27]:
import duckdb

# View top 10 accident-prone states
query = f"""
SELECT State, COUNT(*) AS accident_count
FROM read_csv_auto('{csv_path}', SAMPLE_SIZE=-1)
GROUP BY State
ORDER BY accident_count DESC
LIMIT 10
"""

df_top_states = duckdb.query(query).to_df()
df_top_states


,State,accident_count
0,CA,1741433
1,FL,880192
2,TX,582837
3,SC,382557
4,NY,347960
5,NC,338199
6,VA,303301
7,PA,296620
8,MN,192084
9,OR,179660


**Observations:** California leads by a significant margin in reported accidents, with over 1.7 million incidents—almost double that of the second-ranked state, Florida. Other high-incident states include Texas, South Carolina, and New York, highlighting key regions for further investigation into accident causes and conditions.